<a href="https://colab.research.google.com/github/SubramanyaJ/23CS5PCAIN/blob/main/3A_A_Star_Mismatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def find_empty(board):
    """Find position of empty tile (0)"""
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                return i, j
    return 0, 0

def is_goal(board, goal_board):
    """Check if current state is goal state"""
    return board == goal_board

def copy_board(board):
    """Create a copy of the board"""
    new_board = []
    for i in range(3):
        row = []
        for j in range(3):
            row.append(board[i][j])
        new_board.append(row)
    return new_board

def get_neighbors(board):
    """Get all possible next states"""
    neighbors = []
    empty_row, empty_col = find_empty(board)

    # Try all 4 directions: up, down, left, right
    directions = [(-1, 0, "UP"), (1, 0, "DOWN"), (0, -1, "LEFT"), (0, 1, "RIGHT")]

    for dr, dc, move_name in directions:
        new_row = empty_row + dr
        new_col = empty_col + dc

        # Check if move is within bounds
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            # Create new board by copying current board
            new_board = copy_board(board)

            # Swap empty tile with adjacent tile
            new_board[empty_row][empty_col] = new_board[new_row][new_col]
            new_board[new_row][new_col] = 0

            neighbors.append((new_board, move_name))

    return neighbors

def misplaced_tiles_heuristic(board, goal_board):
    """Count number of tiles in wrong position"""
    count = 0

    for i in range(3):
        for j in range(3):
            if board[i][j] != 0 and board[i][j] != goal_board[i][j]:
                count += 1

    return count

def manhattan_distance_heuristic(board, goal_board):
    """Calculate Manhattan distance for each tile"""
    # Goal positions for each number
    goal_positions = {}
    for i in range(3):
        for j in range(3):
            goal_positions[goal_board[i][j]] = (i, j)

    total_distance = 0

    for i in range(3):
        for j in range(3):
            if board[i][j] != 0:  # Don't calculate for empty tile
                goal_row, goal_col = goal_positions[board[i][j]]
                distance = abs(i - goal_row) + abs(j - goal_col)
                total_distance += distance

    return total_distance

def print_board(board):
    """Print the board nicely"""
    print("┌─────────┐")
    for row in board:
        print("│", end="")
        for cell in row:
            if cell == 0:
                print("   ", end="")
            else:
                print(f" {cell} ", end="")
        print("│")
    print("└─────────┘")

def board_to_string(board):
    """Convert board to string for comparison"""
    result = ""
    for row in board:
        for cell in row:
            result += str(cell)
    return result

def find_min_f_cost(open_list):
    """Find node with minimum f cost in open list"""
    min_f = float('inf')
    min_index = -1

    for i in range(len(open_list)):
        if open_list[i]['f_cost'] < min_f:
            min_f = open_list[i]['f_cost']
            min_index = i

    return min_index

def reconstruct_path(node):
    """Reconstruct path from goal to start"""
    path = []
    current = node

    while current['parent'] is not None:
        path.append({'board': current['board'], 'move': current['move']})
        current = current['parent']

    path.reverse()
    return path

def a_star_solver(start_board, goal_board, heuristic_type):
    """A* algorithm implementation"""

    # Choose heuristic function
    if heuristic_type == 1:
        heuristic_func = misplaced_tiles_heuristic
        heuristic_name = "Misplaced Tiles"
    else:
        heuristic_func = manhattan_distance_heuristic
        heuristic_name = "Manhattan Distance"

    print(f"\nUsing {heuristic_name} heuristic")
    print("="*40)

    # Check if already solved
    if is_goal(start_board, goal_board):
        print("Puzzle already solved!")
        return

    # Initialize open and closed lists
    open_list = []
    closed_list = []

    # Create start node
    start_node = {
        'board': start_board,
        'g_cost': 0,
        'h_cost': heuristic_func(start_board, goal_board),
        'f_cost': 0,
        'parent': None,
        'move': "START"
    }
    start_node['f_cost'] = start_node['g_cost'] + start_node['h_cost']

    open_list.append(start_node)
    nodes_expanded = 0

    while len(open_list) > 0:
        # Find node with minimum f cost
        current_index = find_min_f_cost(open_list)
        current_node = open_list.pop(current_index)
        closed_list.append(current_node)
        nodes_expanded += 1

        # Check if goal reached
        if is_goal(current_node['board'], goal_board):
            print(f"Solution found! Nodes expanded: {nodes_expanded}")
            print(f"Solution length: {current_node['g_cost']} moves\n")

            # Reconstruct and print path
            path = reconstruct_path(current_node)

            print("Solution path:")
            print("Initial state:")
            print_board(start_board)

            for step, state in enumerate(path):
                print(f"\nStep {step + 1}: Move {state['move']}")
                print_board(state['board'])

            return

        # Get neighbors
        neighbors = get_neighbors(current_node['board'])

        for neighbor_board, move in neighbors:
            # Check if neighbor is in closed list
            neighbor_string = board_to_string(neighbor_board)
            in_closed = False

            for closed_node in closed_list:
                if board_to_string(closed_node['board']) == neighbor_string:
                    in_closed = True
                    break

            if in_closed:
                continue

            # Calculate costs
            g_cost = current_node['g_cost'] + 1
            h_cost = heuristic_func(neighbor_board, goal_board)
            f_cost = g_cost + h_cost

            # Check if neighbor is in open list with better cost
            in_open = False
            for open_node in open_list:
                if board_to_string(open_node['board']) == neighbor_string:
                    if g_cost < open_node['g_cost']:
                        open_node['g_cost'] = g_cost
                        open_node['f_cost'] = f_cost
                        open_node['parent'] = current_node
                        open_node['move'] = move
                    in_open = True
                    break

            # Add to open list if not already there
            if not in_open:
                neighbor_node = {
                    'board': neighbor_board,
                    'g_cost': g_cost,
                    'h_cost': h_cost,
                    'f_cost': f_cost,
                    'parent': current_node,
                    'move': move
                }
                open_list.append(neighbor_node)

    print("No solution found!")

def get_puzzle_input():
    """Get puzzle input from user"""
    print("Enter your 3x3 puzzle:")
    print("Use 0 for empty tile")
    print("Enter each row (3 numbers separated by spaces):")

    board = []
    for i in range(3):
        while True:
            try:
                row_input = input(f"Row {i+1}: ").strip()
                row = list(map(int, row_input.split()))

                if len(row) != 3:
                    print("Please enter exactly 3 numbers")
                    continue

                # Check if numbers are valid (0-8)
                valid = True
                for num in row:
                    if num < 0 or num > 8:
                        print("Numbers must be between 0 and 8")
                        valid = False
                        break

                if valid:
                    board.append(row)
                    break

            except ValueError:
                print("Please enter valid integers")

    return board

def main():
    """Main function"""
    print("="*50)
    print("     A* SLIDING PUZZLE SOLVER")
    print("="*50)

    # Get puzzle input
    puzzle = get_puzzle_input()

    print("\nYour puzzle:")
    print_board(puzzle)

    # Get goal state input
    print("\nEnter your 3x3 goal state:")
    goal_puzzle = get_puzzle_input()

    print("\nYour goal state:")
    print_board(goal_puzzle)

    # Choose heuristic
    print("\nChoose heuristic:")
    print("1. Misplaced Tiles")
    print("2. Manhattan Distance")

    while True:
        try:
            choice = int(input("Enter choice (1 or 2): "))
            if choice in [1, 2]:
                break
            else:
                print("Please enter 1 or 2")
        except ValueError:
            print("Please enter a valid number")

    # Solve puzzle
    a_star_solver(puzzle, goal_puzzle, choice)

if __name__ == "__main__":
    main()

     A* SLIDING PUZZLE SOLVER
Enter your 3x3 puzzle:
Use 0 for empty tile
Enter each row (3 numbers separated by spaces):

Your puzzle:
┌─────────┐
│ 1  5  8 │
│ 3  2    │
│ 4  6  7 │
└─────────┘

Enter your 3x3 goal state:
Enter your 3x3 puzzle:
Use 0 for empty tile
Enter each row (3 numbers separated by spaces):

Your goal state:
┌─────────┐
│ 1  2  3 │
│ 4  5  6 │
│ 7  8    │
└─────────┘

Choose heuristic:
1. Misplaced Tiles
2. Manhattan Distance

Using Manhattan Distance heuristic
